In [1]:
import numpy as np
import time
import sys
import random
from threading import Timer
sys.path.append('./utils')
from Reaction import *
from GenerateStrategies import *

In [2]:
money_mode = np.load('./strategies/money_mode.npy')
supply_mode = np.load('./strategies/supply_mode.npy')

In [3]:
stage = 0

In [4]:
# Decide how to survive.
waits = (140, 270, 370, 580)

In [5]:
'''
Decide which army to choose based on given strategies. Quick and cheap armies are favored.
'''
def decide_move(strategy, opmove):
    if opmove == 6:  # 对付Ninja指定用Samurai
        return 4
    if opmove == 7:  # 对付Novice指定用Ninja
        return 6
    if opmove == 13:  # 对付Mage指定用Immortal
        return 12
    if (strategy[1] in (6, 8, 11, 14)) and (strategy[0] not in (6, 8, 9, 11, 12, 14, 15, 16)):
        return strategy[1]

    return strategy[0]

In [6]:
t0 = time.time()
last_move = None
op_last_move = None
allow_surprise = False
surprise_area = (1100, 800, 1200, 860)
defense_area = (720, 800, 900, 860)
turn_time = 24.9  # 每一轮的时间
stage = 0

'''
Stage 0.
'''
while stage == 0:
    t1 = time.time()

    '''
    Decide my move.
    '''
    try:
        opmove = get_opponent_move()
    except:
        continue

    if opmove == 16:  # 一旦对手出High Lord，我就进入stage 1 应对。
        stage = 1
        break

    my_moves = money_mode[opmove-1]
    surprise = surprise_attack(allow_surprise, stage, opmove, surprise_area)  # 前几轮没有压倒性优势，不允许奇袭。对方出Lord，我也出High Lord

    if surprise != False:
        my_move = surprise
    else:
        if t1 - t0 < waits[0]:
            my_move = 1  # 前三轮刷小兵赚钱，反正塔有防御，对面攻不过来
        elif t1 - t0 < waits[1]:
            my_move = random.choice([4, 5, 6])  # 然后刷Samurai/Sword Man/Ninja赚钱，反正塔有防御，对面攻不过来
        elif t1 - t0 < waits[2]:
            strategy = np.extract(my_moves <= 9, my_moves)
            my_move = decide_move(strategy, opmove)
        elif t1 - t0 < waits[3]:  # 中间轮不用Lord
            strategy = np.extract(my_moves <= 14, my_moves)
            my_move = decide_move(strategy, opmove)
            allow_surprise = True
        else:
            strategy = my_moves
            my_move = decide_move(strategy, opmove)
            allow_surprise = True
    
    
    click_area(my_move)
    last_move = my_move
    op_last_move = opmove

    time.sleep(0.01)

'''
Stage 1.
'''
t_0 = time.time() - 0.1
turns = 0
opmoves = [16, 16]

while stage == 1:


    try:
        opmove = get_opponent_move()
    except:
        continue  # 跳过本次循环

    surprise = surprise_attack(allow_surprise, stage, opmove, surprise_area)

    if turns % 2 == 1 or defense(defense_area):
        my_move = 16
        click_area(my_move)
        time.sleep(6)
        click_area(7)
    
    elif surprise != False:  # 奇袭优先
        my_move = surprise

    else:
        if opmove != 16:
            my_moves = supply_mode[opmove-1]
            strategy = np.extract(my_moves <= 14, my_moves)
            for i in strategy:
                if i in (4, 6, 7, 9, 10, 12, 13):  # 找到符合要求的最优军种
                    my_move = i
                    break

        elif opmoves[-2] != 16:  # 对方只出一轮High Lord，我上两轮压制。
            my_move == 16
                
        else:
            my_move = 10  # 没事补点Vampire。

        click_area(my_move)

    turns = turns + 1
    opmoves.append(opmove)
    
    t_1 = time.time()
    time.sleep(turn_time - (t_1 - t_0) % turn_time)

d:\mypyprojects\miragine\./utils\Reaction.py:95: RuntimeWarning: Mean of empty slice.
  x_mean = np.array(x_all).mean()
e:\veighna_studio\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
d:\mypyprojects\miragine\./utils\Reaction.py:96: RuntimeWarning: Mean of empty slice.
  y_mean = np.array(y_all).mean()
